In [1]:
import requests
import pandas as pd
from API_Key import api_key

pages = list(range(1,11))

popular = pd.DataFrame()

for page in pages:
    movieData = pd.DataFrame(requests.get(f'https://api.themoviedb.org/3/movie/popular?api_key={api_key}&language=en-US&page={page}').json()['results'])
    popular = popular.append(movieData)

popular.head()


popular = popular[popular['adult'] == False]

titleNid = popular[['id','title']]

titleNid = titleNid.reset_index(drop = True)
popular.head()

,adult,backdrop_path,genre_ids,id,original_language,original_title,overview,popularity,poster_path,release_date,title,video,vote_average,vote_count
0,False,/cinER0ESG0eJ49kXlExM0MEWGxW.jpg,"[28, 12, 14]",566525,en,Shang-Chi and the Legend of the Ten Rings,Shang-Chi must confront the past he thought he...,7750.478,/1BIoJGKbXjdFDAqUEiA2VHqkK1Z.jpg,2021-09-01,Shang-Chi and the Legend of the Ten Rings,False,7.9,3181
1,False,/dK12GIdhGP6NPGFssK2Fh265jyr.jpg,"[28, 35, 80, 53]",512195,en,Red Notice,An Interpol-issued Red Notice is a global aler...,5461.940,/wdE6ewaKZHr62bLqCn7A2DiGShm.jpg,2021-11-04,Red Notice,False,6.9,1051
2,False,/70nxSw3mFBsGmtkvcs91PbjerwD.jpg,"[878, 28, 12]",580489,en,Venom: Let There Be Carnage,After finding a host body in investigative rep...,4295.722,/rjkmN1dniUHVYAtwuV3Tji7FsDO.jpg,2021-09-30,Venom: Let There Be Carnage,False,6.8,2008
3,False,/u5Fp9GBy9W8fqkuGfLBuuoJf57Z.jpg,"[12, 28, 53]",370172,en,No Time to Die,Bond has left active service and is enjoying a...,4212.055,/iUgygt3fscRoKWCV1d0C7FbM9TP.jpg,2021-09-29,No Time to Die,False,7.6,1927
4,False,/zBkHCpLmHjW2uVURs5uZkaVmgKR.jpg,"[16, 35, 10751]",585245,en,Clifford the Big Red Dog,As Emily struggles to fit in at home and at sc...,3574.383,/tY5jylU5wWaqRs8fmwkJBNzoyXB.jpg,2021-11-10,Clifford the Big Red Dog,False,7.9,150


In [2]:
popular_features = popular[['id','title','release_date','popularity','vote_average','vote_count']]
popular_features = popular_features.rename(columns={
                                        "id":"movie_id",
                                        "popularity":"movie_popularity"
                                        })
popular.head()

,adult,backdrop_path,genre_ids,id,original_language,original_title,overview,popularity,poster_path,release_date,title,video,vote_average,vote_count
0,False,/cinER0ESG0eJ49kXlExM0MEWGxW.jpg,"[28, 12, 14]",566525,en,Shang-Chi and the Legend of the Ten Rings,Shang-Chi must confront the past he thought he...,7750.478,/1BIoJGKbXjdFDAqUEiA2VHqkK1Z.jpg,2021-09-01,Shang-Chi and the Legend of the Ten Rings,False,7.9,3181
1,False,/dK12GIdhGP6NPGFssK2Fh265jyr.jpg,"[28, 35, 80, 53]",512195,en,Red Notice,An Interpol-issued Red Notice is a global aler...,5461.940,/wdE6ewaKZHr62bLqCn7A2DiGShm.jpg,2021-11-04,Red Notice,False,6.9,1051
2,False,/70nxSw3mFBsGmtkvcs91PbjerwD.jpg,"[878, 28, 12]",580489,en,Venom: Let There Be Carnage,After finding a host body in investigative rep...,4295.722,/rjkmN1dniUHVYAtwuV3Tji7FsDO.jpg,2021-09-30,Venom: Let There Be Carnage,False,6.8,2008
3,False,/u5Fp9GBy9W8fqkuGfLBuuoJf57Z.jpg,"[12, 28, 53]",370172,en,No Time to Die,Bond has left active service and is enjoying a...,4212.055,/iUgygt3fscRoKWCV1d0C7FbM9TP.jpg,2021-09-29,No Time to Die,False,7.6,1927
4,False,/zBkHCpLmHjW2uVURs5uZkaVmgKR.jpg,"[16, 35, 10751]",585245,en,Clifford the Big Red Dog,As Emily struggles to fit in at home and at sc...,3574.383,/tY5jylU5wWaqRs8fmwkJBNzoyXB.jpg,2021-11-10,Clifford the Big Red Dog,False,7.9,150


In [3]:
titleNid = titleNid.rename(columns={"id" : 'movie_id'})

In [4]:
titleNid.to_csv('Tables/titleNid.csv',index = False)

In [5]:
# Loop through ids for revnue values
id_list = titleNid['movie_id'].tolist()

revenue_list = []
budget_list = []

for movie in id_list:
    x = requests.get(f'https://api.themoviedb.org/3/movie/{movie}?api_key={api_key}&language=en-US').json()['revenue']
    y = requests.get(f'https://api.themoviedb.org/3/movie/{movie}?api_key={api_key}&language=en-US').json()['budget']
    revenue_list.append(x)
    budget_list.append(y)

revenue_df = pd.DataFrame(
    {'movie_id':id_list,
    'budget': budget_list,
    'revenue':revenue_list
    })

revenue_df.reset_index(drop = True)

,movie_id,budget,revenue
0,566525,150000000,430238384
1,512195,200000000,0
2,580489,110000000,442000000
3,370172,242000000,708000000
4,585245,0,22000000
...,...,...,...
195,198663,34000000,348319861
196,768744,0,38410729
197,841755,0,0
198,337401,200000000,66800000


In [6]:
popular_features

,movie_id,title,release_date,movie_popularity,vote_average,vote_count
0,566525,Shang-Chi and the Legend of the Ten Rings,2021-09-01,7750.478,7.9,3181
1,512195,Red Notice,2021-11-04,5461.940,6.9,1051
2,580489,Venom: Let There Be Carnage,2021-09-30,4295.722,6.8,2008
3,370172,No Time to Die,2021-09-29,4212.055,7.6,1927
4,585245,Clifford the Big Red Dog,2021-11-10,3574.383,7.9,150
...,...,...,...,...,...,...
15,198663,The Maze Runner,2014-09-10,187.018,7.2,14017
16,768744,My Hero Academia: World Heroes' Mission,2021-08-06,186.775,6.9,26
17,841755,Mortal Kombat Legends: Battle of the Realms,2021-08-30,184.977,8.1,184
18,337401,Mulan,2020-09-04,184.493,7.0,5260


In [7]:

Movie_Data = popular_features.merge(revenue_df, on='movie_id', how='inner')
Movie_Data.to_csv('Tables/Movie_Data.csv', index=False)